In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import seaborn as sns
from sklearn import preprocessing
import re

if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = 'c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)

from bs4 import BeautifulSoup
import requests
from datetime import datetime
from datetime import timedelta
import time
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression
from matplotlib import font_manager, rc
font_path = 'C:/Windows/Fonts/NGULIM.TTF'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [119]:
vod_2308 = pd.read_csv('데이터스쿨3차_2308월/�����ͽ���_3��_VOD_2308.csv', encoding='cp949', sep='\t')
vod_2309 = pd.read_csv('데이터스쿨3차_2309월/�����ͽ���_3��_VOD_2309.csv', encoding='cp949', sep='\t')

In [121]:
# 8,9월 데이터 합치기
vod_89 = pd.concat([vod_2308, vod_2309])
vod_89

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227
2,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230829194727
3,61689000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회(22/07/21),TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",1:13,20230804092737
...,...,...,...,...,...,...,...,...,...
5037,60169000,간질간질,키즈,학습,360,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,0:06,20230914211110
5038,66317000,알사탕,키즈,학습,480,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223544
5039,66317000,알사탕,키즈,학습,200,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223210
5040,66900000,월간 아기상어(영어) 01회,키즈,학습,46,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,0:01,20230904120057


In [123]:
vod_89[vod_89['asset_nm'] == '영웅연대']

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
4225,65931000,영웅연대,영화,무협,5388,청나라 말기. 영국은 중국의 문호를 열기 위해 전함 21척을 진격시키는 한편 일본인...,"하상,문동준,유개,오일가",1:33,20230925131539


In [107]:
# 특수문자 정리
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\(.*$', '', regex=True)
vod_89

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",1:13,20230804092737
...,...,...,...,...,...,...,...,...,...
5037,60169000,간질간질,키즈,학습,360,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,0:06,20230914211110
5038,66317000,알사탕,키즈,학습,480,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223544
5039,66317000,알사탕,키즈,학습,200,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223210
5040,66900000,월간 아기상어 01회,키즈,학습,46,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,0:01,20230904120057


In [108]:
# 특수문자 정리
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\(.*$', '', regex=True)

In [110]:
# 회차 삭제
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\d+회$', '', regex=True)
vod_89['asset_nm'] = vod_89['asset_nm'].str.replace(r'\d+회\.', '', regex=True)

In [113]:
vod_89[vod_89['asset_nm'].str.contains('회')][:30]

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
392,65526000,심야괴담회,TV 연예/오락,기타,9,괴담 투어를 하러 온 더 보이즈 주연과 큐. 익숙한 장소에서 느낀 낯선 무언가의 존...,"김구라,김숙,이이경,솔라",1:04,20230820001412
396,66189000,심야괴담회,TV 연예/오락,기타,3900,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230804041838
397,62613000,심야괴담회,TV 연예/오락,기타,3900,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230817194503
398,64495000,심야괴담회,TV 연예/오락,기타,1449,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230803092518
399,66309000,심야괴담회,TV 연예/오락,기타,3900,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230803180753
400,66189000,심야괴담회,TV 연예/오락,기타,0,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230803225648
401,60984000,심야괴담회,TV 연예/오락,기타,3900,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230803202821
402,65526000,심야괴담회,TV 연예/오락,기타,701,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230820091415
403,66309000,심야괴담회,TV 연예/오락,기타,360,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230806202010
404,62212000,심야괴담회,TV 연예/오락,기타,1521,15년 전 필리핀에 이민을 갔던 사연자가 도망치듯 한국으로 귀국하게 된 사연을 소개...,"김구라,김숙,황제성,김아영",1:05,20230823002720


In [61]:
# vod_89에서 asset_nm, ct_cl 추출
vod_list = vod_89[['asset_nm','ct_cl']].drop_duplicates(subset='asset_nm')

In [62]:
vod_list

,asset_nm,ct_cl
0,그것이알고싶다,TV 시사/교양
4,꼬리에꼬리를무는그날이야기,TV 시사/교양
9,인간극장,TV 시사/교양
11,오은영 리포트 결혼 지옥,TV 시사/교양
13,실화탐사대,TV 시사/교양
...,...,...
5035,엄마가 화났다,키즈
5036,핑크퐁 자동차 동화,키즈
5037,간질간질,키즈
5040,월간 아기상어,키즈


In [64]:
vod_list[vod_list['asset_nm'].str.contains('회')]

,asset_nm,ct_cl
392,심야괴담회,TV 연예/오락
630,회장님네 사람들,TV 연예/오락
4302,상류사회,영화
5327,19회 고창 1부,우리동네
5328,20회 고창 2부,우리동네
5331,21회 전라남도 여수 2부,우리동네
5530,재활용 음악회에 오세요,키즈
2567,"암련, 귤생회남",TV드라마
4643,극장판 주술회전 0,영화


In [65]:
vod_list['ct_cl'].unique()

array(['TV 시사/교양', 'TV 연예/오락', 'TV드라마', 'TV애니메이션', '다큐', '영화', '우리동네',
       '키즈', '공연/음악', '스포츠'], dtype=object)

In [3]:
# unique 모아놓은 csv
vod_data = pd.read_csv('vod_data.csv', encoding='cp949')
vod_data

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,지니강이 플러스 시즌7,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...
1,라바 키즈동요 Part3,NaN,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...
2,지니강이 플러스 시즌9,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.
3,헤이지니 럭키강이 Part1,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.
4,우당탕탕 에디의 실험실,NaN,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...
...,...,...,...,...,...,...,...,...,...
3633,엄마가 화났다,NaN,-,NaN,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...
3634,핑크퐁 자동차 동화,NaN,핑크퐁,NaN,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...
3635,간질간질,NaN,-,NaN,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
3636,월간 아기상어,NaN,아기상어,NaN,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...


In [7]:
# 더 나은 크롤링을 위해 vod 이름에 있는 - 를 blank로 대체한 열 생성
vod_data['vod_name'] = vod_data['name'].str.replace("-", " ")

In [4]:
contents_2308 = pd.read_csv('데이터스쿨3차_2308월/�����ͽ���_3��_������_2308.csv', encoding='cp949', sep='\t')
contents_2309 = pd.read_csv('데이터스쿨3차_2309월/�����ͽ���_3��_������_2309.csv', encoding='cp949', sep='\t')
vod_2308 = pd.read_csv('데이터스쿨3차_2308월/�����ͽ���_3��_VOD_2308.csv', encoding='cp949', sep='\t')
vod_2309 = pd.read_csv('데이터스쿨3차_2309월/�����ͽ���_3��_VOD_2309.csv', encoding='cp949', sep='\t')

In [8]:
vod_data

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7
1,라바 키즈동요 Part3,NaN,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3
2,지니강이 플러스 시즌9,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,지니강이 플러스 시즌9
3,헤이지니 럭키강이 Part1,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.,헤이지니 럭키강이 Part1
4,우당탕탕 에디의 실험실,NaN,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,우당탕탕 에디의 실험실
...,...,...,...,...,...,...,...,...,...,...
3633,엄마가 화났다,NaN,-,NaN,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,엄마가 화났다
3634,핑크퐁 자동차 동화,NaN,핑크퐁,NaN,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁 자동차 동화
3635,간질간질,NaN,-,NaN,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,간질간질
3636,월간 아기상어,NaN,아기상어,NaN,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,월간 아기상어


In [11]:
# 크롤링 정보 - 영화 : 네이버, 이외 : 다음
for idx, row in vod_data.iterrows():
    if pd.isna(row['Summary']):
        name = row[-1]
        category = row[4]
        print(name)
        print(category)
        if category == '영화':
            movie_name = ' '.join(['영화', name])
            movie_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_name}"
            movie_web_page = requests.get(movie_url).text
            m_soup = BeautifulSoup(movie_web_page, 'html.parser')
            html = m_soup.select('body > div > div > div > div > div > div > div > div > div > p')
            if html == []:
                html = m_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
                if html == []:
                    movie_name = str(' '.join([name, '정보']))
                    movie_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_name}"
                    movie_web_page = requests.get(movie_url).text
                    m_soup = BeautifulSoup(movie_web_page, 'html.parser')
                    html = m_soup.select('body > div > div > div > div > div > div > div > div > div > p')
                    if html == []:
                        html = m_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
                        if html == []:
                            print()
                        else:
                            m_smry = str(html[0]).split(">")[1].split("<")[0]
                            vod_data.at[idx, 'Summary'] = m_smry
                            print(m_smry)
                            time.sleep(5)  # 5초 동안 대기
                    else:
                            m_smry = str(html[0]).split(">")[1].split("<")[0]
                            vod_data.at[idx, 'Summary'] = m_smry
                            print(m_smry)
                            time.sleep(5)  # 5초 동안 대기
                else:
                            m_smry = str(html[0]).split(">")[1].split("<")[0]
                            vod_data.at[idx, 'Summary'] = m_smry
                            print(m_smry)
                            time.sleep(5)  # 5초 동안 대기
            else:
                m_smry = str(html[0]).split(">")[1].split("<")[0]
                vod_data.at[idx, 'Summary'] = m_smry
                print(m_smry)
                time.sleep(5)  # 5초 동안 대기
        elif 'TV' in category or '다큐' in category or '키즈' in category:
            asset_name = ' '.join([name, '소개'])
            asset_url = f"https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={asset_name}"
            asset_web_page = requests.get(asset_url).text
            a_soup = BeautifulSoup(asset_web_page, 'html.parser')
            a_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd')
            if a_list == []:
                print()
            else:
                try:
                    a_smry = str(a_list).split('class="highlighting">')[1].split('<')[0]
                    print(a_smry)
                    vod_data.at[idx, 'Summary'] = a_smry
                except:
                    try:
                        a_smry = str(a_list[0]).split(">")[1].split("<")[0]
                        print(a_smry)
                        vod_data.at[idx, 'Summary'] = a_smry
                    except:
                        print()
        

가십 #그녀가 알고 싶은 진짜 OO
TV드라마
대형 출판사가 운영하는 인터넷 뉴스의 편집부원이 거짓과 진실이 뒤섞인 세계 속에서 분투하는 모습을 그린 드라마
소용없어 거짓말
TV드라마
거짓말이 들리는 능력 때문에 사람을 믿지 못하는 여자가 누구도 결백을 믿어주지 않는 살인 용의자와 얽히며 진실을 파헤치는 미스터리 로맨스
화피의 진애무회
TV드라마
서로 먹고 먹히는 세력 다툼이 전쟁으로 이어지는 시대적인 혼란 속에서, 각자의 신분, 출생, 환경을 뛰어넘어 자신만의 사랑을 위해 목숨까지 아끼지 않는 사랑 이야기를 그린 드라마
왕좌의 게임 4
TV드라마
판타지 소설 '얼음과 불의 노래'를 드라마화한 작품
미스터 션샤인
TV드라마
신미양요(1871년)때 군함에 승선해 미국에 떨어진 한 소년이 미국 군인 신분으로, 자신을 버린 조국인 조선으로 와 주둔하며 벌어지는 일을 그린 드라마
이로운 사기
TV드라마
공감불능 사기꾼과 과공감 변호사, 너무나 다른 두 사람의 절대악을 향한 복수극이자 짜릿한 공조 사기극
환혼 화면해설방송본
TV드라마

고검기담
TV드라마
일생의 스승을 찾아 여행을 떠난 락천이는 그 여정 속에서 전사 리우잉, 승려 하이칙, 아완을 만나게 되고 이 네 사람은 인간 세계를 파괴하려 나타난 어둠의 세력과 맞서 마침내 세상을 구하게 되면서 벌어지는 이야기
도시 전설의 여자
TV드라마
도시 전설 오타쿠인 형사 오토나시 츠키코가 동료들과 함께 도시 전설과 관련된 살인 사건을 해결하는 형사 드라마
다이버: 특수잠입반
TV드라마
악을 응징하기 위해 비밀리에 만들어진 특수 잠입 수사반의 활약을 그린 서스펜스 드라마
머독 미스터리 14
TV드라마
1895년 캐나다 토론토를 배경으로 하며 형사 머독이 해박한 과학지식으로 사건을 해결해나가는 과정을 통해, 당시 캐나다 사회의 어두운 부분들을 드러내는 드라마
대무당:무림고수
TV드라마
주체는 조카 주윤문을 끌어내리고 황위를 찬탈하지만, 주윤문이 실종되자 불안에 빠지고, 목흔은 주체를 처단하기 위해 주체의 딸 상녕 공주와 혼인을

의적 소십일랑이 쫓기던 중 무림 전설로 내려오는 신비한 보물을 둘러싼 싸움에 휘말리게 되는 이야기
응답하라 1988
TV드라마
쌍팔년도 쌍문동, 한 골목 다섯 가족의 왁자지껄 코믹 가족 드라마
수호지
TV드라마
진정한 영웅을 꿈꾸는 남자들의 이야기
오!마이 보스! 사랑은 별책으로
TV드라마
패션잡지 편집부를 배경으로 펼쳐지는 로맨틱 코미디 드라마
망황기 
TV드라마
천부적인 자질을 지녀 무공에 재능을 가진 기씨 부족 영주의 아들 기녕이 멸족의 변을 당하고 복수의 길을 떠나며 겹겹의 무험과 장애를 경험하면서 성장하는 이야기를 그린 드라마
우리는 사랑이 서툴러
TV드라마
친한 친구와 온천 여행을 하던 중 우연히 두 남자를 만나 시작되는 러브 스토리
드래프트 킹
TV드라마
최고의 선수를 영입하기 위한 두 스카우트맨의 투지와 활약을 그린 드라마
삼국지
TV드라마
4백 여 년을 이어온 한 제국 정권이 급속히 몰락해 나가면서 거대한 소용돌이가 눈앞에 닥쳐오게 되고, 그 후로 백 여 년에 이르는 난세에 돌입하게 되는데 지금까지 전해 내려오는 ‘삼국’시대의 대한 이야기를 그린 드라마 
창란결
TV드라마
난꽃 정령 소란화와 월족 수장인 동방청창의 운명과 사랑을 그린 이야기
뿅뿅 지구오락실2
TV 연예/오락
지구로 도망간 달나라 토끼를 잡기 위해 뭉친 4명의 용사가 시공간을 넘나들며 펼치는 버라이어티 예능 프로그램
정글의 법칙 in 멕시코
TV 연예/오락
병만족의 정글 생존기를 담은 프로그램
대국민토크쇼 안녕하세요
TV 연예/오락
시청자의 고민을 소개하고 사연의 주인공을 스튜디오로 초대해 이야기를 나누는 토크콘서트 프로그램
뮤직뱅크
TV 연예/오락
다양한 장르의 대중가요 및 최신 음악 정보를 전달하는 고품격 가요 쇼 프로그램
인생술집
TV 연예/오락
한 잔 술, 한 모금 인생. 성인들의 가장 평범한 저녁. 퇴근길 삼삼오오 술자리를 통해 '스타'의 이야기가 아닌, '사람'의 이야기를 나누는 프로그램
빈집살래 시즌3   수리수리 마을수리
TV 연예/오락
마을 전체를 바꿀 빈집의 놀

아이들이 태어나서 처음으로 마주하는 히어로 ‘슈퍼맨’의 육아 도전기
벌거벗은 세계사
TV 연예/오락
전 세계 곳곳을 온택트로 둘러보며 각 나라의 명소를 살펴보고, 다양한 관점에서 우리가 몰랐던 세계의 역사를 파헤치는 프로그램
형따라 마야로 : 아홉 개의 열쇠
TV 연예/오락
마야 문명의 비밀의 열쇠를 찾아 떠나는 생활 밀착 문명 어드벤처
MBC 이즈 백
TV 연예/오락
MZ 세대의 추억 속 프로그램부터 현재진행형 대세 프로그램까지! 입담 만랩 2MC 박미선 &amp; 홍현희와 함께 쇼, 예능, 드라마 등 MBC를 통해 방송됐던 인기 프로그램들을 모아보는 프로그램
미생
TV 연예/오락
바둑만이 인생의 모든 것이었던 주인공이 프로입단에 실패한 후, 냉혹한 현실에 던져지면서 벌어지는 이야기를 그린 드라마
2019코미디빅리그4쿼터하이라이트
TV 연예/오락

신서유기 시즌4
TV 연예/오락
리얼막장 모험활극, 신서유기! 지금까지 들어본 적도 없고, 차마 볼 수도 없었던 리얼 막장 예능 프로그램
황금어장
TV 연예/오락
가수, 배우, 작가 등 각 계층의 유명 인사나 스타를 초대해 진실되면서도 날카로운 얘기를 나눈다
1박2일 시즌4
TV 연예/오락
우리가 알고 있는, 우리가 알지 못하는 아름다운 우리나라. 유쾌한 다섯 남자와 함께 1박 2일의 여행을 떠나는 프로그램
한끼 줍쇼
TV 연예/오락
하루를 살아가는 원동력, 소통의 매개체이기도 했던 우리네 저녁 밥상. 평범한 가정, 국민들의 저녁 속으로 들어가 저녁 한 끼 나누며 오늘을 살아가는 우리들의 모습을 엿보고자하는 프로그램
살림하는 남자들 2
TV 연예/오락
신세대부터 중년 그리고 노년 까지! 스타 살림남들의 리얼 살림기
알토란
TV 연예/오락
매일 먹는 집 밥에 대한 대한민국 최고 셰프들의 제안! 따라 하고 싶은 한 끼 [알토란]
걸어서 환장 속으로
TV 연예/오락
“정말 환장하겠네” 피를 나눈 사이지만 피 터지게 싸운다. 닮은 듯 다른 스타 가족들의 좌충우돌 해외여행! 환상 혹은 환장이 될 패밀리 월드투어
생존게임 코

속도의 시대에 잃어버리고 살았던 동네의 아름다움, 오아시스 같은 사람들을 보물찾기하듯 동네의 숨은 매력을 재발견하며 팍팍한 삶에 따뜻한 위안을 전하는 도시 기행 다큐멘터리 프로그램
장바구니 집사들
TV 시사/교양
식재료가 담긴 장바구니를 후원하는 청년 먹거리 솔루션 프로젝트
다큐인사이트 한우랩소디
TV 시사/교양

동물극장 단짝
TV 시사/교양
반려동물과 사람 사이의 우정과 사랑을 그린 반려동물다큐
돈워리스쿨
TV 시사/교양
재야의 경제 고수 유튜버 슈카와 신사임당, 스브스뉴스의 대표 연반인 재재, 그리고 김윤상 주시은 아나운서가 월 200만 원 내외로 사회생활을 시작하는 2030 세대의 눈높이에 맞춰 이들의 돈에 대한 고민과 애환에 도움을 줄 정보를 제공하는 프로그램
다큐인사이트 미래전쟁
TV 시사/교양

2020 역사저널 그날
TV 시사/교양

스트레이트
TV 시사/교양
12명의 기자들이 탐사 취재를 해서 전달하는 형식의 심층 취재와 보도로 다양한 사회적 이슈를 성역 없이 끝까지 추적해 진실을 밝힐 신개념 탐사기획 프로그램
PD수첩
TV 시사/교양
'시대의 정직한 목격자'가 되기 위한 성역 없는 취재를 지향하는 심층 탐사 보도 프로그램
이웃집 찰스
TV 시사/교양
익숙한 세상을 떠나 낯선 한국 땅으로 온 외국인들이 바라본 한국의 모습과 그 속에서 소통하기 위해 노력하는 과정 속에서 우리 사회의 이면을 들여다보고 그들과의 진정한 교감을 시도하는 신개념 리얼리티 프로그램
장애인의 날 특집 봄날의 기적
TV 시사/교양
저소득가정 장애인들이 적기에 치료를 받을 수 있도록 후원을 통해 도울 수 있는 프로그램
2022 창업프로젝트 스타트업 서바이벌
TV 시사/교양
대한민국 창업 DNA의 발굴을 위한 프로젝트! 미래 대한민국 경제를 책임지는 아이디어 人들의 불꽃 튀는 경쟁 프로그램
인간극장
TV 시사/교양
결코 평범하지 않은 삶의 이야기를 가진 보통 사람들, 그리고 사회적인 명성을 가진, 이른바 잘나가는 사람들의 아주 평범한 이야기를 담은 다큐미니시리즈
이웃집 찰스
T

암울한 시대를 살아가야만 했던 풍운아들의 이야기를 김두한을 중심으로 그들의 인생과 사랑, 아픔을 그려낸 드라마 
휴먼다큐 사노라면
TV 시사/교양
때로는 생명과도 같은 나의 핏줄, 때로는 원수보다 못한 징글징글한 전생의 업, 가족! 소소하지만 나름의 삶을 지탱하는 '위대한 일상'을 통해 시청자들의 공감하는 '삶의 비법'을 풀어가는 프로그램
나는자연인이다
TV 시사/교양
원시의 삶 속 대자연의 품에서 저마다의 사연을 간직한 채 자연과 동화되어 욕심없이 살아가는 이들의 이야기를 담은 프로그램
강석우의 종점여행
TV 시사/교양
국민 배우 강석우가 버스 종점으로 떠나며 겪는 일화를 다루는 여행 프로그램
한문철의 블랙박스 리뷰
TV 시사/교양
알면 살고, 모르면 죽는다! ‘대한민국 No.1 차통령’ 한문철 변호사가 블랙박스 속 숨겨진 정보를 파헤치는 엄지척 교통안전 프로젝트
강석우의 종점여행 2
TV 시사/교양
국민 배우 강석우가 버스 종점으로 떠나며 겪는 일화를 다루는 여행 프로그램
차이나는 클라스
TV 시사/교양
특별 과외수업 형태로 강연자와 학생 간의 거리감을 좁혀 좀 더 밀도 있는 지식 · 정보 전달과 자유로운 질문을 통한 소통형 스토리텔링 수업으로 진행하는 프로그램
슈퍼맨이돌아왔다
TV 연예/오락
아이들이 태어나서 처음으로 마주하는 히어로 ‘슈퍼맨’의 육아 도전기
엄지의 제왕
TV 연예/오락
그동안 잘 알려지지 않았던 생활 속 놀라운 건강 정보를 누구나 쉽게 따라할 수 있도록 소개하는 프로그램
최강야구
TV 연예/오락
Win or Nothing. 오직 승리만을 추구하는 사상 최강의 야구팀이 탄생했다. “우리보다 최강인 팀은 절대 없을 겁니다” 프로야구팀에 대적할만한 11번째 구단 ‘최강 몬스터즈’와 전국의 야구 강팀이 펼치는 양보 없는 대결을 그린 프로그램
대화의 희열
TV 연예/오락
지금 당장 만나고 싶은 '단 한 사람'과의 뜨거운 대화! 단독 토크쇼의 명맥을 묵직하게 이어가는 토크멘터리
짠당포
TV 연예/오락
짠내나던 시절, 소중한 물건을 전당포에 맡긴 스

...


재하리우
TV드라마
현대에선 누구에게나 무시받던 리우는 과거로 타임슬립하여 관아의 하인이 된다. 관아들의 비웃음을 받던 리우는 문학적 재능을 발휘하여 사람들을 놀라게하고, 무관과 거상의 집안에 눈에 들어 두 집의 데릴사위가 되는 이야기
스프레드 시트
TV드라마
이혼 후, 두 아이의 엄마이자 늘 바쁜 로렌은 많은 남자들과 하룻밤을 보낸다. 최고의 섹스 파트너를 찾기 위해 자신만의 스프레드 시트를 만들기 시작하는데…
하선생적연연불망
TV드라마
교통사고로 말문을 닫은 아들과 까칠하고 차가운 아빠가 따뜻한 마음을 가진 아동심리 전문의와 만나게 되면서 치유를 받게 되는 로맨스 드라마
절배수심당 : 사랑의 맛
TV드라마
인기 걸그룹 캔디 소녀의 리더로 스타덤에 올랐던 탕쑤는 출처를 알 수 없는 감독과의 추문으로, 남자친구는 물론 연예계에서도 버림받는다. 음식 배달하며 근근이 먹고 살던 중에 우연히 타임 엔터테인먼트 황푸줴와 엮이며 그의 식모가 되는 이야기를 그린 드라마
명불허전
TV드라마
침을 든 조선 최고의 한의사 허임과 메스를 든 현대 의학 신봉자 외과의 연경이 400년을 뛰어넘어 펼치는 조선 왕복 메디활극
미자무강
TV드라마
후당의 혼란스러운 시기 아버지에게 버림받고 기루에 팔리게 되고, 일이 잘못 되어 신비로운 곳인 궤획성에 들어가 완메이라는 이름을 얻게 된 주인공의 이야기를 그린 드라마
미자무강
TV드라마
후당의 혼란스러운 시기 아버지에게 버림받고 기루에 팔리게 되고, 일이 잘못 되어 신비로운 곳인 궤획성에 들어가 완메이라는 이름을 얻게 된 주인공의 이야기를 그린 드라마
봉귀사시가 
TV드라마
어려움 속에 서로를 지키며 함께 나라와 백성을 위해 분투하게 되는 그들의 달콤한 고전 로맨스 드라마
연희공략: 건륭황제의여인
TV드라마
한 소녀가 언니의 죽음에 대한 진상을 알기 위해 자금성으로 들어가 궁녀가 되고, 궁정에서 발생하는 각종 어려움을 헤쳐 나가고 이후 건륭 성세에 일조하는 귀비가 되는 이야기
유리미인살 
TV드라마
어릴 때부터 육감에 문제가 있어 인간이 느

후당의 혼란스러운 시기 아버지에게 버림받고 기루에 팔리게 되고, 일이 잘못 되어 신비로운 곳인 궤획성에 들어가 완메이라는 이름을 얻게 된 주인공의 이야기를 그린 드라마
연희공략: 건륭황제의여인
TV드라마
한 소녀가 언니의 죽음에 대한 진상을 알기 위해 자금성으로 들어가 궁녀가 되고, 궁정에서 발생하는 각종 어려움을 헤쳐 나가고 이후 건륭 성세에 일조하는 귀비가 되는 이야기
절배수심당 : 사랑의 맛
TV드라마
인기 걸그룹 캔디 소녀의 리더로 스타덤에 올랐던 탕쑤는 출처를 알 수 없는 감독과의 추문으로, 남자친구는 물론 연예계에서도 버림받는다. 음식 배달하며 근근이 먹고 살던 중에 우연히 타임 엔터테인먼트 황푸줴와 엮이며 그의 식모가 되는 이야기를 그린 드라마
38 사기동대
TV드라마
세금 징수 공무원과 사기꾼이 합심하여, 편법으로 부를 축적하고 상습적으로 탈세를 저지르는 악덕 체납자들에게 세금을 징수하는 통쾌한 스토리를 다룬 드라마
선협 로맨스: 옥소령 
TV드라마
이단 세력을 처단하기 위해 인간 세상에 내려온 봉래의 상선, 단목취. 계봉부의 미남 포졸 전안이 만나서 범죄를 해결하고 사랑을 찾아가는 이야기를 그린 드라마
철인왕후
TV드라마
불의의 사고로 대한민국 대표 허세남 영혼이 깃들어 '저 세상 텐션'을 갖게 된 중전 김소용과 두 얼굴의 임금 철종 사이에서 벌어지는 영혼가출 스캔들
삼생삼세 침상서
TV드라마
천족과 신선들 사이에서 일어나는 사건들을 그린 판타지 로맨스 드라마
절배수심당 : 사랑의 맛
TV드라마
인기 걸그룹 캔디 소녀의 리더로 스타덤에 올랐던 탕쑤는 출처를 알 수 없는 감독과의 추문으로, 남자친구는 물론 연예계에서도 버림받는다. 음식 배달하며 근근이 먹고 살던 중에 우연히 타임 엔터테인먼트 황푸줴와 엮이며 그의 식모가 되는 이야기를 그린 드라마
무림유협기
TV드라마

백앤아 남매튜브 2
TV애니메이션

백앤아 남매튜브 2
TV애니메이션

로빈TV 미니게임 PART4
TV애니메이션

백앤아 남매튜브 2
TV애니메이션

짱구는 못말려 23기
TV애

가족의탄생
영화
누가 보면 연인 사이라 오해할 만큼 다정한, 친구 같고 애인 같은 남매 미라(문소리)와 형철(엄태웅). 인생이 자유로운 형철은 5년 동안 소식 없다 불현듯 누나 미라를 찾아온다. 인생이 조금은 흐릿한 20살 연상녀인 무신(고두심)과 함께.. 똑 부러지는 인생을 꿈꾸던 미라는 사랑하는 동생 형철 그리고 동생이 사랑하는 여인 무신과의 아슬아슬, 어색한 동거를 시작하는데.. 한편, 리얼리스트 선경(공효진)은 로맨티스트 엄마 매자(김혜옥)때문에 인생이 조용할 날이 없다. 사랑이라면 만사 오케이인 엄마의 뒤치다꺼리 하다 보니 이리저리 치인 기억에 사랑이 마냥 좋지만은 않은 선경. 남자친구 준호(류승범)와의 애정전선에 낀 먹구름도 맑게 개일 기미를 보이지 않는다. 딸의 연애가 위태위태한데 매자는 왜 또 선경을 찾으시는지.. 그리고 그 놈의 사랑 때문에 인생이 편할 날 없는 경석(봉태규)과 채현(정유미)이 있다. 얼굴도 예쁘고 맘도 예쁜 채현이 넘치는 사랑을 주위 사람들에게 너무 나누어주다 보니 정작 남자친구는 애정결핍증에 걸리고 만 기구한 커플이다. 이건 아니다 싶은 경석. 참고 참다 둘 사이에 강수를 놓기로 하는데..과연 채현이 그 수에 걸려들까? 하루가 멀다 하고 웬~수처럼 으르렁대는 이들.. 사랑만으로도 복잡한데 이 7명은 여기저기서 또 얽히고 설킨 스캔들로 인생 들썩이기 일쑤다. 어쩌다 저렇게 엮이는지, 살짝 피곤해지려고 할 때. 꿈에도 생각지 못한 하나의 비밀이 이들에게 다가오는데. 사랑에, 스캔들에, 바람 잘 날 없는 이들 과연 찬란한 행복이 탄생할 수 있을까?
더 킹
영화
무소불위 권력을 쥐고 폼 나게 살고 싶었던 태수는 우여곡절 끝에 권력의 설계자 한강식을 만나 핵심 라인을 타고 승승장구 하게 된다 정권이 교체되는 중요한 시기, 새로운 판을 짜며 기회를 노리던 이들 앞에 예상치 못한 위기가 닥치는데… 대한민국이 속 시원하게 뒤집힌다!
두근두근 내 인생
영화
한 때 헛발 왕자로 불리던 태권도 유망주 ‘대수’와 아이돌을 꿈꾸던 당찬 성격의 ‘미

세상에서 가장 빠른 비행기 ‘호기’ 장난감 악당에 맞서 지구를 구해줘! 누구보다 빠른 스피드로 사람들에게 꿈과 희망을 배달하는 ‘호기’! 장난감이 인기가 없어져 버린 세상에서 악당 ‘빌리윌리’는 유튜브 스타들을 납치해 우주로 날려버릴 무시무시한 음모를 세운다. 100만 유튜버 ‘다인’은 납치된 엄마를 구하기 위해 ‘호기’와 함께 ‘빌리윌리’를 찾아나서는데…! 과연, ‘호기’는 ‘맥시멈 스피드’로 지구를 구할 수 있을까?
보스베이비 2
영화
베이비 주식회사의 레전드 보스 베이비에서 인생 만렙 CEO가 된 ‘테드’. 베이비인 줄 알았던 조카 ‘티나’가 알고 보니 베이비 주식회사 소속 임원으로 밝혀진다. 뉴 보스 베이비 ‘티나’의 지시로 다시 베이비로 돌아간 ‘테드’와 형 ‘팀’에게 주어진 시간은 48시간! 세상을 구하기 위한 미션을 무사히 완수할 수 있을 것인가?
미니언즈2
영화
세계 최고의 슈퍼 악당을 꿈꾸는 미니보스 ‘그루’와 그를 따라다니는 미니언들. 어느 날 그루는 최고의 악당 조직 ‘빌런6’의 마법 스톤을 훔치는데 성공하지만 뉴페이스 미니언 ‘오토’의 실수로 스톤을 잃어버리고 빌런6에게 납치까지 당한다. 미니보스를 구하기 위해 잃어버린 스톤을 되찾아야 하는 ‘오토’, 그리고 쿵푸를 마스터해야 하는 ‘케빈’, ‘스튜어트’, ‘밥’! 올여름 극장가를 점령할 MCU(미니언즈 시네마틱 유니버스)가 돌아온다!
졸업반
영화
프랑스 유학을 꿈꾸며 순수미술 전공 졸업작품을 준비하고 있는 ‘주희(강진아)’는 뛰어난 실력과 함께 외모를 겸비한, 모든 이들의 선망의 대상이다. 웹툰 작가 지망생인 같은 학과 동기 ‘정우(이주승)’는 얼음공주로 소문난 ‘주희’를 좋아하지만, 그녀에게 다가갈 수조차 없다. 그러던 중 ‘정우’는 낮에는 대학생으로, 밤에는 텐프로로 이중생활을 하는 ‘주희’의 은밀한 비밀을 알게 되고, 비밀을 공유하게 된 두 사람은 점점 가까워진다. 넘치는 돈과 우수한 성적으로 졸업을 원하는 ‘주희’를 ‘정우’가 물심양면 돕게 되는 것. 하지만 ‘정우’의 절친인 ‘동

카운트
영화
1988년 올림픽 금메달리스트지만 1998년 지금은 평범한 고등학교 선생인 ‘시헌’(진선규). 선수 생활 은퇴 후 남은 건 고집뿐, 모두를 킹받게 하는 마이웨이 행보로 주변 사람들의 속을 썩인다. 그러던 어느 날, 우연히 참석한 대회에서 뛰어난 실력에도 불구하고 승부 조작으로 기권패를 당한 ‘윤우’(성유빈)를 알게 된 ‘시헌’은 복싱부를 만들기로 결심한다. 아내 ‘일선’(오나라)의 열렬한 반대와, ‘교장’(고창석)의 끈질긴 만류도 무시한 채, ‘시헌’은 독기만 남은 유망주 ‘윤우’와 영문도 모른 채 레이더망에 걸린 ‘환주’(장동주), ‘복안’(김민호)을 데리고 본격적인 훈련에 돌입하기 시작하는데...! 쓰리, 투, 원! 긍정 파워 풀충전! 그들만의 가장 유쾌한 카운트가 시작된다
요절복통 일망타진
영화
반공의식이 투철한 김도달과 허풍길은 애인을 만나기 위해 상경하던 중 늑대라는 별명의 간첩을 알게 되어 그가 소지하고 있던 암호문을 얻어 경찰에 신고한다. 그리하여 본격적인 대간첩작전이 전개되어 마침내 간첩일당이 일망타진되고 그들 두 사람은 각기 애인을 만나 정부에서 보상하는 상금을 타가지고 시골로 내려간다.
장르만 로맨스
영화
매일매일 버라이어티한 그 작가의 사생활 개봉박두! 쿨내진동 이혼부부 일촉즉발 비밀커플 주객전도 스승제자 알쏭달쏭 이웃사촌 평범하지 않은 로맨스로 얽힌 이들의 사생활이 밝혀진다!
경남 통영 2부
우리동네
핑크퐁 이솝이야기
키즈

핑크퐁 율동동요
키즈

엘리가간다 시즌4
키즈

엘리가간다 시즌4
키즈

핑크퐁 모양송
키즈

핑크퐁 율동동요
키즈

핑크퐁 뮤지컬 공룡동화
키즈

콩순이 율동교실 5기
키즈

라바 키즈동요 Part2
키즈

엘리가간다 시즌4
키즈

엘리가간다 시즌4
키즈

핑크퐁 바다동물동요
키즈

뽀로로 지구탐험동화
키즈

함께 놀자 뽀로로
키즈

핑크퐁 율동동요
키즈

핑크퐁 뮤지컬 공룡동화
키즈

엘리가간다 시즌4
키즈

캐리의 마녀 슬라임 연구소
키즈

헤이지니 럭키강이 시즌10 스페셜
키즈

먹놀잠 프로젝트
키

In [27]:
vod_data.to_csv('vod_data_addSummary.csv', encoding='utf-8')

In [14]:
vod_data

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7
1,라바 키즈동요 Part3,NaN,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3
2,지니강이 플러스 시즌9,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,지니강이 플러스 시즌9
3,헤이지니 럭키강이 Part1,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.,헤이지니 럭키강이 Part1
4,우당탕탕 에디의 실험실,NaN,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,우당탕탕 에디의 실험실
...,...,...,...,...,...,...,...,...,...,...
3633,엄마가 화났다,NaN,-,NaN,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,엄마가 화났다
3634,핑크퐁 자동차 동화,NaN,핑크퐁,NaN,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁 자동차 동화
3635,간질간질,NaN,-,NaN,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,간질간질
3636,월간 아기상어,NaN,아기상어,NaN,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,월간 아기상어


In [20]:
vod_data['Summary'].fillna(vod_data['SMRY'], inplace=True)

In [26]:
vod_data.head(9)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7
1,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3
2,지니강이 플러스 시즌9,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,지니강이 플러스 시즌9
3,헤이지니 럭키강이 Part1,지니가 초대형 볼풀장에 놀러간 이야기.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.,헤이지니 럭키강이 Part1
4,우당탕탕 에디의 실험실,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,우당탕탕 에디의 실험실
5,달려라 캐리,캐리가 세차장 아르바이트생이 되었어요! 엄격한 선배님으로 돌아온 돌쇠와 함께 버블 ...,-,NaN,키즈,기타,키즈,0:19,캐리가 세차장 아르바이트생이 되었어요! 엄격한 선배님으로 돌아온 돌쇠와 함께 버블 ...,달려라 캐리
6,라바 키즈동요 Part2,FEELINGS. Why are you so happy? i am happy bec...,"옐로우,레드",NaN,키즈,기타,키즈,0:02,FEELINGS. Why are you so happy? i am happy bec...,라바 키즈동요 Part2
7,캐리tv 댄스cool,오늘은 친구들이 #댄스쿨 챌린지에 참여해 준 영상들을 루시와 스텔라가 직접 리뷰해 ...,-,NaN,키즈,기타,키즈,0:14,오늘은 친구들이 #댄스쿨 챌린지에 참여해 준 영상들을 루시와 스텔라가 직접 리뷰해 ...,캐리tv 댄스cool
8,안녕자두야 시즌5,"80년대 소녀기를 보낸 요즘 10대들의 엄마 어렸을 적 이야기로, 온가족이 볼 수 ...",최자두,NaN,키즈,애니메이션,키즈,0:14,자두와 앵두 (하). 모함글 하나로 순식간에 대역죄인으로 찍힌 자두 가족. 자두와 ...,안녕자두야 시즌5


In [35]:
vod_data.drop_duplicates(subset='name', keep='first')

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7
1,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3
2,지니강이 플러스 시즌9,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,지니강이 플러스 시즌9
3,헤이지니 럭키강이 Part1,지니가 초대형 볼풀장에 놀러간 이야기.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.,헤이지니 럭키강이 Part1
4,우당탕탕 에디의 실험실,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,우당탕탕 에디의 실험실
...,...,...,...,...,...,...,...,...,...,...
3633,엄마가 화났다,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,-,NaN,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,엄마가 화났다
3634,핑크퐁 자동차 동화,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁,NaN,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁 자동차 동화
3635,간질간질,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,NaN,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,간질간질
3636,월간 아기상어,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,NaN,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,월간 아기상어


In [30]:
vod_data.head(20)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7
1,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3
2,지니강이 플러스 시즌9,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,지니강이 플러스 시즌9
3,헤이지니 럭키강이 Part1,지니가 초대형 볼풀장에 놀러간 이야기.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.,헤이지니 럭키강이 Part1
4,우당탕탕 에디의 실험실,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,우당탕탕 에디의 실험실
5,달려라 캐리,캐리가 세차장 아르바이트생이 되었어요! 엄격한 선배님으로 돌아온 돌쇠와 함께 버블 ...,-,NaN,키즈,기타,키즈,0:19,캐리가 세차장 아르바이트생이 되었어요! 엄격한 선배님으로 돌아온 돌쇠와 함께 버블 ...,달려라 캐리
6,라바 키즈동요 Part2,FEELINGS. Why are you so happy? i am happy bec...,"옐로우,레드",NaN,키즈,기타,키즈,0:02,FEELINGS. Why are you so happy? i am happy bec...,라바 키즈동요 Part2
7,캐리tv 댄스cool,오늘은 친구들이 #댄스쿨 챌린지에 참여해 준 영상들을 루시와 스텔라가 직접 리뷰해 ...,-,NaN,키즈,기타,키즈,0:14,오늘은 친구들이 #댄스쿨 챌린지에 참여해 준 영상들을 루시와 스텔라가 직접 리뷰해 ...,캐리tv 댄스cool
8,안녕자두야 시즌5,"80년대 소녀기를 보낸 요즘 10대들의 엄마 어렸을 적 이야기로, 온가족이 볼 수 ...",최자두,NaN,키즈,애니메이션,키즈,0:14,자두와 앵두 (하). 모함글 하나로 순식간에 대역죄인으로 찍힌 자두 가족. 자두와 ...,안녕자두야 시즌5
9,뽀스타그램2,"타요 특집: 멋진 내 친구들! 경찰차도, 소방차도, 모두모두 멋진 내 친구야! [해...",뽀로로와 친구들,NaN,키즈,애니메이션,키즈,0:22,"타요 특집: 멋진 내 친구들! 경찰차도, 소방차도, 모두모두 멋진 내 친구야! [해...",뽀스타그램2


In [31]:
vod_data.reset_index(drop=True)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7
1,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3
2,지니강이 플러스 시즌9,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,지니강이 플러스 시즌9
3,헤이지니 럭키강이 Part1,지니가 초대형 볼풀장에 놀러간 이야기.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.,헤이지니 럭키강이 Part1
4,우당탕탕 에디의 실험실,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,우당탕탕 에디의 실험실
...,...,...,...,...,...,...,...,...,...,...
3633,엄마가 화났다,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,-,NaN,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,엄마가 화났다
3634,핑크퐁 자동차 동화,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁,NaN,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁 자동차 동화
3635,간질간질,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,NaN,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,간질간질
3636,월간 아기상어,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,NaN,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,월간 아기상어


In [32]:
vod_data.to_csv('vod_data_addSummary.csv', encoding='utf-8')

In [ ]:
# 만드는 중

# # 영화 감독 : 네이버, 영화 외 제작 : 다음
# for idx, row in vod_data.iterrows():
#     if pd.isna(row[3]):    
#         name = row[-1]
#         category = row[4]
#         print(name)
#         print(category)
#         if category == '영화':
#             m_director_keyword = ' '.join(['영화', name, '출연진'])
#             m_director_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={m_director_keyword}"
#             m_director_web_page = requests.get(m_director_url).text
#             m_director_soup = BeautifulSoup(m_director_web_page, 'html.parser')
#             info = m_director_soup.select('body > div > div > div > div > div > div > div > div > div > div > div > div > ul > li > div > div > div > strong > span')
#             if info == []:
#                 html = m_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
#                 if html == []:
#                     movie_name = str(' '.join([name, '정보']))
#                     movie_url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={movie_name}"
#                     movie_web_page = requests.get(movie_url).text
#                     m_soup = BeautifulSoup(movie_web_page, 'html.parser')
#                     html = m_soup.select('body > div > div > div > div > div > div > div > div > div > p')
#                     if html == []:
#                         html = m_soup.select('body > div > div > div > div > div > div > div > div > div > div > span')
#                         if html == []:
#                             print()
#                         else:
#                             m_smry = str(html[0]).split(">")[1].split("<")[0]
#                             vod_data.at[idx, 'Summary'] = m_smry
#                             print(m_smry)
#                             time.sleep(5)  # 5초 동안 대기
#                     else:
#                             m_smry = str(html[0]).split(">")[1].split("<")[0]
#                             vod_data.at[idx, 'Summary'] = m_smry
#                             print(m_smry)
#                             time.sleep(5)  # 5초 동안 대기
#                 else:
#                             m_smry = str(html[0]).split(">")[1].split("<")[0]
#                             vod_data.at[idx, 'Summary'] = m_smry
#                             print(m_smry)
#                             time.sleep(5)  # 5초 동안 대기
#             else:
#                 m_smry = str(html[0]).split(">")[1].split("<")[0]
#                 vod_data.at[idx, 'Summary'] = m_smry
#                 print(m_smry)
#                 time.sleep(5)  # 5초 동안 대기
#         elif 'TV' in category or '다큐' in category or '키즈' in category:
#             asset_name = ' '.join([name, '소개'])
#             asset_url = f"https://search.daum.net/search?nil_suggest=btn&w=tot&DA=SBC&q={asset_name}"
#             asset_web_page = requests.get(asset_url).text
#             a_soup = BeautifulSoup(asset_web_page, 'html.parser')
#             a_list = a_soup.select('body > div > div > main > div > div > div > div > div > div > div > div > div > div > dl > dd')
#             if a_list == []:
#                 print()
#             else:
#                 try:
#                     a_smry = str(a_list).split('class="highlighting">')[1].split('<')[0]
#                     print(a_smry)
#                     vod_data.at[idx, 'Summary'] = a_smry
#                 except:
#                     try:
#                         a_smry = str(a_list[0]).split(">")[1].split("<")[0]
#                         print(a_smry)
#                         vod_data.at[idx, 'Summary'] = a_smry
#                     except:
#                         print()
        

In [51]:
vod_data[vod_data['director'].isna() == True]

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
0,지니강이 플러스 시즌7,친구들 안녕! 지니와 강이와 함께 신나는 장난감 놀이와 두근두근 대결놀이까지! 함께...,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...,지니강이 플러스 시즌7
1,라바 키즈동요 Part3,CLEAN UP SONG. Clean up Clean up everybody cle...,-,NaN,키즈,기타,키즈,0:02,CLEAN UP SONG. Clean up Clean up everybody cle...,라바 키즈동요 Part3
2,지니강이 플러스 시즌9,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,지니가 화장품으로 슬라임을 만들어요. / 슈퍼히어로 강이! 문을 열면 춤을 춰요.,지니강이 플러스 시즌9
3,헤이지니 럭키강이 Part1,지니가 초대형 볼풀장에 놀러간 이야기.,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:08,지니가 초대형 볼풀장에 놀러간 이야기.,헤이지니 럭키강이 Part1
4,우당탕탕 에디의 실험실,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,"에디,뽀로로,통통이,크롱,포비,루피,패티",NaN,키즈,기타,키즈,0:04,말랑말랑 스퀴시 만들기! 에디와 함께 스퀴시를 만들어 볼까요? 오늘은 루피를 위해 ...,우당탕탕 에디의 실험실
...,...,...,...,...,...,...,...,...,...,...
3633,엄마가 화났다,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,-,NaN,키즈,학습,키즈,0:08,인기작가 최숙희의 그림책. 아이들에게 건네는 화해와 위로의 메시지! 아이와 엄마 사...,엄마가 화났다
3634,핑크퐁 자동차 동화,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁,NaN,키즈,학습,키즈,0:04,Lil Reds First Fire(용감한 소방차 뚜뚜와 뿌뿌). 불이 난 곳이면 ...,핑크퐁 자동차 동화
3635,간질간질,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,NaN,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,간질간질
3636,월간 아기상어,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,NaN,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,월간 아기상어


In [54]:
vod_data[vod_data['name'].str.contains('극장판')]

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY,vod_name
392,밥만 잘 사주는 이상한 이사님 극장판,네 번의 생을 모두 기억하고 살아낸 유담과 그와 연애해야만 생을 유지할 수 있는 신...,"박영운,박정우,장유성",NaN,영화,멜로,영화,2:06,네 번의 생을 모두 기억하고 살아낸 유담과 그와 연애해야만 생을 유지할 수 있는 신...,밥만 잘 사주는 이상한 이사님 극장판
406,슬램덩크 극장판 1기-전국제패를 향해,전국대회 예선전 대전상대로 만난 진무고교. 진무의 슈팅가드 오대일이 전반 내내 상북...,"강수진,김승준,이정구,정미숙",NaN,영화,애니메이션,영화,0:40,서비스 해지 시까지 시청 가능. 전국 대회 예선전 대전 상대로 만난 진무고교. 진무...,슬램덩크 극장판 1기 전국제패를 향해
410,짱구는못말려 극장판8기,"신 짱은 신작 액션가면 영화를 호화 여객선을 타며 미리 볼수 있다는 광고를 보고, ...","짱구,짱구 아빠,짱구 엄마,짱아",NaN,영화,애니메이션,영화,NaN,액션가면과 짱구의 정글 대모험. 휴가를 맞아 유람선여행을 떠난 짱구가족. 그러나 원...,짱구는못말려 극장판8기
411,아따맘마 극장판,“우리 어떻게 되는 거야??”아줌마틱한 ‘엄마’와 소심한 고등학생 딸 ‘아리’. 어...,"이미자,이계윤,손종환,홍범기,오리카사 후미코,오가타 켄이치,사카구치 다이스케,오리카...",NaN,영화,애니메이션,영화,1:35,서비스 해지 시까지 시청 가능. 아줌마틱한 엄마와 소심한 고등학생 딸 아리. 어느 ...,아따맘마 극장판
414,뽀로로 극장판 보물섬 대모험,전설 속 보물을 찾아 떠난 뽀로로와 친구들은 우연히 손에 넣은 지도를 따라 신비의 ...,"이선,이미자,김현지,홍소영,정미숙,김서영,김환진,남도형,정승욱,이장원,황창영",NaN,영화,애니메이션,영화,1:19,전설 속 보물을 찾아 떠난 뽀로로와 친구들! 우연히 손에 넣은 지도 덕분에 신비의 ...,뽀로로 극장판 보물섬 대모험
418,[극장판] 왕실교사 하이네,그란츠라이히 왕국의 왕자 4명은 차기 국왕을 목표로 왕실교사 하이네 비트겐슈타인의 ...,"우에다 케이스케,아사토 유우야,아다치 유우토,히로세 다이스케",NaN,영화,애니메이션,영화,1:00,그란츠라이히 왕국의 왕자 4명은 차기 국왕을 목표로 왕실교사 하이네 비트겐슈타인의 ...,[극장판] 왕실교사 하이네
419,극장판 안녕 자두야,‘자두’는 제주도 가족여행에서 만난 엄친딸 제주 소녀 ‘전복’이가 못마땅하다. 게다...,"자두,엄마,미미,애기,꼬꼬,자두렐라 엄마,아빠,민지,누룽지,은희",NaN,영화,애니메이션,영화,NaN,어린이날을 맞아 가족들과 놀이공원에 놀러 간 최자두. 동생들과 놀이기구를 타던 자두...,극장판 안녕 자두야
436,은하철도 999 - 극장판,999호를 타고 떠났던 여행으로부터 1년 뒤. 한때 프로메슘 여왕을 쓰러트린 영웅으...,"노자와 마사코,이케다 마사코,타지마 레이코,키모츠키 카네타,토야마 케이",NaN,영화,애니메이션,영화,2:08,인간의 몸을 기계로 바꿔 영원한 생명을 살수 있게 된 먼 미래. 기계백작에게 엄마를...,은하철도 999 극장판
441,극장판 도라에몽: 진구의 신공룡,진구는 공룡 엑스포 화석 발굴 체험에서 발견한 화석을 공룡알이라고 굳게 믿는다. 도...,"윤아영,김정아,조현정,이현주",NaN,영화,애니메이션,영화,1:50,진구는 공룡 엑스포 화석 발굴 체험에서 발견한 화석을 공룡알이라고 굳게 믿는다. 도...,극장판 도라에몽: 진구의 신공룡
444,극장판 영원의 831,전대미문의 대재해에 의해 전 세계가 곤욕을 치르고 있는 현대. 동경에서 신문장학생으...,"사이토 소마,M.A.O,오키츠 카즈유키,히카사 요코",NaN,영화,애니메이션,영화,1:45,전대미문의 대재해에 의해 전 세계가 곤욕을 치르고 있는 현대. 동경에서 신문장학생으...,극장판 영원의 831


In [55]:
vod_data = vod_data.applymap(lambda x: x.replace('[', '').replace(']', ''))

AttributeError: 'float' object has no attribute 'replace'